In [4]:
import math
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from gpt import GPT2
from transformers import pipeline, set_seed

In [8]:
hf_model = GPT2LMHeadModel.from_pretrained('gpt2', resume_download=None)
hf_model.eval()

gpt2_model = GPT2.from_pretrained()
gpt2_model.eval()

print("Loaded models")


Loaded models
